In [ ]:
import numpy as np
from time import ctime
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.core.base import Constraint as pyo_constraint
from pyomo.core.base import Var as pyo_vars

def taskload_array_to_dict(taskload_array):
    
    out_dict = {}
    
    for i in range(taskload_array.shape[0]):
        for j in range(taskload_array.shape[1]):
            out_dict[(i, j)] = taskload_array[i, j]
            
    return out_dict


def agent_availability_dict_processer(taskload_parameters, agent_availability_time_intervals_dict):
    
    number_time_intervals = taskload_parameters['number_time_intervals']
    
    out_dict = {}
    
    for k, v in agent_availability_time_intervals_dict.items():
        for ti in range(number_time_intervals):
            if v[0] <= ti <= v[1]:
                out_dict[(k, ti)] = 1
            else:
                out_dict[(k, ti)] = 0
                
    return out_dict


def objective_function_minimise_taskload(model):
    
    return sum(sum(model.r[ g, t]  for g in model.G) for t in model.T)


def objective_function_minimise_taskload_and_group_reconfigurations(model):
    
    return sum(sum(model.r[ g, t]  for g in model.G) for t in model.T) + \
           model.group_any_reconfig_weight * sum(sum(model.x[g, t] for g in model.G) for t in model.T_minus) + \
           model.group_job_reconfig_weight * sum(sum(sum(model.w[j, g, t] for j in model.J) for g in model.G) \
            for t in model.T_minus)


def objective_function_minimise_taskload_and_group_and_agent_reconfigurations(model):
    
    return sum(sum(model.r[ g, t]  for g in model.G) for t in model.T) + \
           model.group_any_reconfig_weight * sum(sum(model.x[g, t] for g in model.G) for t in model.T_minus) + \
           model.group_job_reconfig_weight * sum(sum(sum(model.w[j, g, t] for j in model.J) for g in model.G) \
                                             for t in model.T_minus) + \
           model.group_agent_reconfig_weight * sum(sum(sum(model.y[a, g, t] for a in model.A) for g in model.G) \
                                               for t in model.T_minus)

def constraint_link_p_and_q_variables(model, g, t):
    return sum(model.p[j, g, t] for j in model.J) <= model.number_jobs * sum(model.q[a, g, t] for a in model.A)

def constraint_link_p_and_q_variables_2(model, g, t):
    return sum(model.p[j, g, t] for j in model.J) >= sum(model.q[a, g, t] for a in model.A)

def constraint_link_p_and_r_variables(model, g, t):
    return sum(model.p[j, g, t] for j in model.J) <= model.number_jobs * model.r[g, t]

def constraint_link_p_and_r_variables_2(model, g, t):
    return sum(model.p[j, g, t] for j in model.J) >= model.r[g, t]

def constraint_link_q_and_s_variables(model, a, t):
    return sum(model.q[a, g, t] for g in model.G) == model.s[a, t] 

def constraint_total_coverage(model, j, t):
    return sum(model.p[j, g, t] for g in model.G) == 1
    
def constraint_one_agent_per_group(model, g, t):
    return sum(model.q[a, g, t] for a in model.A) <= 1
    

def constraint_one_group_per_agent(model, a, t):
    return sum(model.q[a, g, t] for g in model.G) <= 1

def constraint_max_tasks(model, g, t):    
    return sum(model.taskload[j, t] * model.p[j, g ,t] for j in model.J) <= model.taskload_limit

def constraint_limit_agent_working_hours(model, a, t):
    return model.s[a, t] <= model.agent_availability[a, t]

def constraint_max_working_time(model, a, t):    
    return (model.max_working_time - sum(model.s[a, t2] for t2 in range(t, t + model.max_working_time)) >= 
           model.s[a, t + model.max_working_time])

def constraint_min_working_time(model, a, t):
    
    if t == 0:
        return ((model.min_working_time-1) * model.s[a, t] <= 
                sum(model.s[a, t2] for t2 in range(t + 1, t + model.min_working_time)))
    if t == model.number_time_intervals - model.min_working_time:
        return ((model.min_working_time-1) * model.s[a, t] >= 
                sum(model.s[a, t2] for t2 in range(t + 1, t + model.min_working_time)))
    else:
        return (model.min_working_time * (model.s[a, t - 1] + (1-model.s[a, t])) >= 
                (model.min_working_time - 1 - sum(model.s[a, t2] for t2 in range(t + 1, t + model.min_working_time))))
    
def constraint_min_break_time(model, a, t):

    return (model.min_break_time * ((1 - model.s[a, t - 1]) + model.s[a, t]) >= 
            sum(model.s[a, t2] for t2 in range(t + 1, t + model.min_break_time)))   

def constraint_break_symmetry(model, g, t):
    return model.r[g, t] >= model.r[g + 1, t] 

def constraint_link_p_and_w_variables_1(model, j, g, t):
    return model.w[j, g, t] >= model.p[j, g, t] - model.p[j, g, t-1]

def constraint_link_p_and_w_variables_2(model, j, g, t):
    return model.w[j, g, t] >= model.p[j, g, t-1] - model.p[j, g, t] 

def constraint_link_p_and_w_variables_3(model, j, g, t):
    return model.w[j, g, t] <= model.p[j, g, t] + model.p[j, g, t-1]

def constraint_link_p_and_w_variables_4(model, j, g, t):
    return model.w[j, g, t] <= 2 - model.p[j, g, t] - model.p[j, g, t-1] 

def constraint_link_w_and_x_variables(model, g, t):
    return sum(model.w[j, g, t] for j in model.J) <= model.number_jobs * model.x[g, t]

def constraint_link_q_and_y_variables_1(model, a, g, t):
    return model.y[a, g, t] >= model.q[a, g, t] - model.q[a, g, t-1]

def constraint_link_q_and_y_variables_2(model, a, g, t):
    return model.y[a, g, t] >= model.q[a, g, t-1] - model.q[a, g, t]

def constraint_link_q_and_y_variables_3(model, a, g, t):
    return model.y[a, g, t] <= model.q[a, g, t] + model.q[a, g, t-1]

def constraint_link_q_and_y_variables_4(model, a, g, t):
    return model.y[a, g, t] <= 2 - model.q[a, g, t] - model.q[a, g, t-1]


def build_model(taskload_parameters, agent_parameters, taskload_array, number_groups,
                group_any_reconfig_weight, group_job_reconfig_weight,
                group_agent_reconfig_weight):
    
    agent_availability_time_intervals_dict = agent_parameters['agent_availability_time_intervals_dict']
    max_working_time = agent_parameters['max_working_time']
    min_working_time = agent_parameters['min_working_time']
    min_break_time = agent_parameters['min_break_time']
    number_agents = agent_parameters['number_agents']
    taskload_limit = agent_parameters['taskload_limit']
    number_time_intervals = taskload_parameters['number_time_intervals']
    number_jobs = taskload_parameters['number_jobs']
    
    # define model
    model = pyo.ConcreteModel()
    
    # define parameters
    model.taskload_limit = pyo.Param(initialize=taskload_limit, 
                                     within=pyo.NonNegativeReals)
    
    model.number_time_intervals = pyo.Param(initialize=number_time_intervals, 
                                            within=pyo.NonNegativeReals)
    
    model.number_agents = pyo.Param(initialize=number_agents, 
                                         within=pyo.NonNegativeReals)
    
    model.number_jobs = pyo.Param(initialize=number_jobs, 
                                  within=pyo.NonNegativeReals)
    
    model.number_groups = pyo.Param(initialize=number_groups, 
                                  within=pyo.NonNegativeReals)
    
    model.max_working_time = pyo.Param(initialize=max_working_time, 
                                       within=pyo.NonNegativeReals)
    
    model.min_working_time = pyo.Param(initialize=min_working_time, 
                                       within=pyo.NonNegativeReals)
    
    model.min_break_time = pyo.Param(initialize=min_break_time, 
                                       within=pyo.NonNegativeReals)
    
    model.group_any_reconfig_weight = pyo.Param(initialize=group_any_reconfig_weight, 
                                                within=pyo.NonNegativeReals)
    
    model.group_job_reconfig_weight = pyo.Param(initialize=group_job_reconfig_weight, 
                                                within=pyo.NonNegativeReals)
    
    model.group_agent_reconfig_weight = pyo.Param(initialize=group_agent_reconfig_weight, 
                                                within=pyo.NonNegativeReals)
    
    # define model sets
    model.T = pyo.RangeSet(0, model.number_time_intervals-1, 1, dimen=1, ordered=True)
    model.A = pyo.RangeSet(0, model.number_agents-1, dimen=1)
    model.J = pyo.RangeSet(0, model.number_jobs-1, dimen=1)
    model.G = pyo.RangeSet(0, model.number_groups-1, dimen=1)
    model.G_minus = pyo.RangeSet(0, model.number_groups-2, dimen=1)
    model.T_minus = pyo.RangeSet(1, model.number_time_intervals-1, 1, dimen=1, ordered=True)
    model.T_MaxWork = pyo.RangeSet(0, model.number_time_intervals - model.max_working_time-1, 
                                   1, dimen=1, ordered=True)
    model.T_MinWork = pyo.RangeSet(0, model.number_time_intervals - model.min_working_time, 
                                   1, dimen=1, ordered=True)
    model.T_MinBreak = pyo.RangeSet(1, model.number_time_intervals - model.min_break_time, 1, dimen=1, ordered=True)
    
    # add data to model
    model.taskload = pyo.Param(model.J, model.T, within=pyo.NonNegativeReals, 
                               initialize=taskload_array_to_dict(taskload_array), default=0)
    model.agent_availability = pyo.Param(model.A, model.T, within=pyo.NonNegativeReals, 
                                         initialize=agent_availability_dict_processer(taskload_parameters, 
                                                          agent_availability_time_intervals_dict), 
                                         default=0)
    
    # define decision variables
    model.p = pyo.Var(model.J, model.G, model.T, within=pyo.Binary)
    model.q = pyo.Var(model.A, model.G, model.T, within=pyo.Binary)
    model.r = pyo.Var(model.G, model.T, within=pyo.Binary)
    model.s = pyo.Var(model.A, model.T, within=pyo.Binary)
    
    model.w = pyo.Var(model.J, model.G, model.T_minus, within=pyo.Binary)
    model.x = pyo.Var(model.G, model.T_minus, within=pyo.Binary)
    
    model.y = pyo.Var(model.A, model.G, model.T_minus, within=pyo.Binary)
    
    model.objective_function_minimise_taskload_and_group_and_agent_reconfigurations \
    = pyo.Objective(rule=objective_function_minimise_taskload_and_group_and_agent_reconfigurations,sense=pyo.minimize)
    
    model.constraint_link_p_and_q_variables = pyo.Constraint(model.G, model.T, 
                                                              rule=constraint_link_p_and_q_variables)
    model.constraint_link_p_and_q_variables_2 = pyo.Constraint(model.G, model.T, 
                                                              rule=constraint_link_p_and_q_variables_2)
    model.constraint_link_p_and_r_variables = pyo.Constraint(model.G, model.T, 
                                                              rule=constraint_link_p_and_r_variables)
    model.constraint_link_p_and_r_variables_2 = pyo.Constraint(model.G, model.T, 
                                                              rule=constraint_link_p_and_r_variables_2)
    model.constraint_link_q_and_s_variables = pyo.Constraint(model.A, model.T, 
                                                              rule=constraint_link_q_and_s_variables)
    model.constraint_total_coverage = pyo.Constraint(model.J, model.T, 
                                                              rule=constraint_total_coverage)
    model.constraint_one_agent_per_group = pyo.Constraint(model.G, model.T, 
                                                            rule=constraint_one_agent_per_group)
    model.constraint_one_group_per_agent = pyo.Constraint(model.A, model.T, 
                                                            rule=constraint_one_group_per_agent)
    model.constraint_max_tasks = pyo.Constraint(model.G, model.T, 
                                                rule=constraint_max_tasks)
    model.constraint_limit_agent_working_hours = pyo.Constraint(model.A, model.T, 
                                                                rule=constraint_limit_agent_working_hours)
    model.constraint_max_working_time = pyo.Constraint(model.A, model.T_MaxWork, 
                                                       rule=constraint_max_working_time)
    model.constraint_min_working_time = pyo.Constraint(model.A, model.T_MinWork, 
                                                       rule=constraint_min_working_time)
    model.constraint_min_break_time = pyo.Constraint(model.A, model.T_MinBreak, 
                                                     rule=constraint_min_break_time)
    model.constraint_break_symmetry = pyo.Constraint(model.G_minus, model.T, 
                                                rule=constraint_break_symmetry)
    
    model.constraint_link_p_and_w_variables_1 = pyo.Constraint(model.J, model.G, model.T_minus, 
                                                               rule=constraint_link_p_and_w_variables_1)
    model.constraint_link_p_and_w_variables_2 = pyo.Constraint(model.J, model.G, model.T_minus, 
                                                               rule=constraint_link_p_and_w_variables_2)
    model.constraint_link_p_and_w_variables_3 = pyo.Constraint(model.J, model.G, model.T_minus, 
                                                               rule=constraint_link_p_and_w_variables_3)
    model.constraint_link_p_and_w_variables_4 = pyo.Constraint(model.J, model.G, model.T_minus, 
                                                               rule=constraint_link_p_and_w_variables_4)
    model.constraint_link_w_and_x_variables = pyo.Constraint(model.G, model.T_minus, 
                                                               rule=constraint_link_w_and_x_variables)
    
    model.constraint_link_q_and_y_variables_1 = pyo.Constraint(model.A, model.G, model.T_minus, 
                                                               rule=constraint_link_q_and_y_variables_1)
    model.constraint_link_q_and_y_variables_2 = pyo.Constraint(model.A, model.G, model.T_minus, 
                                                               rule=constraint_link_q_and_y_variables_2)
    model.constraint_link_q_and_y_variables_3 = pyo.Constraint(model.A, model.G, model.T_minus, 
                                                               rule=constraint_link_q_and_y_variables_3)
    model.constraint_link_q_and_y_variables_4 = pyo.Constraint(model.A, model.G, model.T_minus, 
                                                               rule=constraint_link_q_and_y_variables_4)
    
    return model


def solve_instance(model, solver, solver_filepath, max_run_time=600, ratio_gap=0.1
                   , tee=False, warmstart=False, logfile_name = 'logfile.txt'):
    
    print('Run start time: ' + str(ctime()))

    # Use solvers on neos server
    if solver_filepath == 'neos':
        solver_manager = pyo.SolverManagerFactory('neos')
        opt = SolverFactory(solver)
        if solver == 'cplex':
            opt.set_options('mipgap=' + str(ratio_gap))
            opt.set_options('timelimit=' + str(max_run_time))
            opt.set_options('mipdisplay=' + str(3))
            opt.set_options('nodefile=' + str(2))
            opt.set_options('treememory=' + str(10000))
        elif solver == 'mosek':
            opt.set_options('MSK_DPAR_MIO_REL_GAP_CONST=' + str(ratio_gap))
            opt.set_options('MSK_DPAR_OPTIMIZER_MAX_TIME=' + str(max_run_time))
        results = solver_manager.solve(model, opt=opt, keepfiles=True, tee=tee)

    # Use local solver
    else:
        if solver == 'glpk':
            opt = SolverFactory(solver, executable=solver_filepath)
            opt.set_options('tmlim=' + str(max_run_time))
            results = opt.solve(model, tee=tee)
        elif solver == 'cbc':
            opt = SolverFactory(solver, executable=solver_filepath)
            opt.set_options('sec=' + str(max_run_time))
            opt.set_options('ratioGap=' + str(ratio_gap))
            if logfile_name == '':
                results = opt.solve(model, tee=tee, warmstart=warmstart)
            else:
                results = opt.solve(model, logfile=logfile_name, tee=tee, warmstart=warmstart)
        else:
            raise ValueError(f'Solver {solver} not supported')
            
    print('Run finish time: ' + str(ctime()))        
    
    return model, results